In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class Ticker:
    def __init__(self, symbol, start_date, end_date):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date

    def fetch_data(self):
        return yf.download(self.symbol, start=self.start_date, end=self.end_date)

class Strategy:
    def __init__(self, data):
        self.data = data

    def generate_signals(self):
        raise NotImplementedError("Subclasses should implement this!")

class MACDStrategy(Strategy):
    def __init__(self, data, short_window=12, long_window=26, signal_window=9):
        super().__init__(data)
        self.short_window = short_window
        self.long_window = long_window
        self.signal_window = signal_window

    def calculate_macd(self):
        self.data['EMA12'] = self.data['Close'].ewm(span=self.short_window, adjust=False).mean()
        self.data['EMA26'] = self.data['Close'].ewm(span=self.long_window, adjust=False).mean()
        self.data['MACD'] = self.data['EMA12'] - self.data['EMA26']
        self.data['Signal'] = self.data['MACD'].ewm(span=self.signal_window, adjust=False).mean()
        self.data['Histogram'] = self.data['MACD'] - self.data['Signal']
        return self.data

    def generate_signals(self):
        self.data = self.calculate_macd()
        self.data['Buy_Signal'] = ((self.data['MACD'] > self.data['Signal']) &
                                   (self.data['MACD'].shift(1) <= self.data['Signal'].shift(1))).astype(int)
        self.data['Sell_Signal'] = ((self.data['MACD'] < self.data['Signal']) &
                                    (self.data['MACD'].shift(1) >= self.data['Signal'].shift(1))).astype(int)
        return self.data

class BollingerBandsStrategy(Strategy):
    def __init__(self, data, window=20, num_sd=2):
        super().__init__(data)
        self.window = window
        self.num_sd = num_sd

    def calculate_bollinger_bands(self):
        self.data['SMA'] = self.data['Close'].rolling(window=self.window).mean()
        self.data['std'] = self.data['Close'].rolling(window=self.window).std()
        self.data['Upper Band'] = self.data['SMA'] + (self.data['std'] * self.num_sd)
        self.data['Lower Band'] = self.data['SMA'] - (self.data['std'] * self.num_sd)
        return self.data

    def generate_signals(self):
        self.data = self.calculate_bollinger_bands()
        self.data['Buy_Signal'] = (self.data['Close'] < self.data['Lower Band']).astype(int)
        self.data['Sell_Signal'] = (self.data['Close'] > self.data['Upper Band']).astype(int)
        return self.data

class BuyAndHoldStrategy(Strategy):
    def generate_signals(self):
        self.data['Buy_Signal'] = 0
        self.data['Sell_Signal'] = 0
        self.data.loc[self.data.index[0], 'Buy_Signal'] = 1  # Buy at the beginning
        self.data.loc[self.data.index[-1], 'Sell_Signal'] = 1  # Sell at the end
        return self.data

class Backtester:
    def __init__(self, data, initial_capital=100000, transaction_cost=0.001):
        self.data = data
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        self.risk_free_rate = self.fetch_risk_free_rate()

    def fetch_risk_free_rate(self):
        treasury_yield = yf.Ticker("^TNX").history(period="1d")
        if not treasury_yield.empty:
            return treasury_yield['Close'].iloc[-1] / 100
        else:
            return 0.0428

    def backtest(self):
        capital = self.initial_capital
        position = 0
        cash = capital
        portfolio_value = []
        bought_shares = 0
        sold_shares = 0
        buy_prices = []
        sell_prices = []

        for i in range(len(self.data)):
            if self.data['Buy_Signal'].iloc[i] == 1 and cash > 0:
                shares_bought = cash / self.data['Close'].iloc[i]
                position += shares_bought
                bought_shares += shares_bought
                buy_prices.append(self.data['Close'].iloc[i])
                cash = 0
                cash -= self.transaction_cost * (shares_bought * self.data['Close'].iloc[i])
            elif self.data['Sell_Signal'].iloc[i] == 1 and position > 0:
                cash += position * self.data['Close'].iloc[i]
                sold_shares += position
                sell_prices.append(self.data['Close'].iloc[i])
                position = 0
                cash -= self.transaction_cost * (sold_shares * self.data['Close'].iloc[i])
            portfolio_value.append(cash + position * self.data['Close'].iloc[i])

        self.data['Portfolio_Value'] = portfolio_value
        net_profit_loss = cash + position * self.data['Close'].iloc[-1] - self.initial_capital
        total_return = (net_profit_loss / self.initial_capital) * 100

        profit = {'Final Portfolio Value': portfolio_value[-1], 'Net Profit/Loss': net_profit_loss}

        returns = pd.Series(portfolio_value).pct_change().dropna()
        excess_returns = returns - self.risk_free_rate / 252
        sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(252)
        downside_deviation = returns[returns < 0].std()
        sortino_ratio = excess_returns.mean() / downside_deviation * np.sqrt(252)
        max_drawdown = (pd.Series(portfolio_value).cummax() - pd.Series(portfolio_value)).max()
        calmar_ratio = (returns.mean() * 252) / max_drawdown

        metrics = {
            'Initial Capital': self.initial_capital,
            'Net Profit/Loss': net_profit_loss,
            'Total Return (%)': total_return,
            'Total Shares Bought': bought_shares,
            'Total Shares Sold': sold_shares,
            'Buy Prices': buy_prices,
            'Sell Prices': sell_prices,
            'Sharpe Ratio': sharpe_ratio,
            'Sortino Ratio': sortino_ratio,
            'Maximum Drawdown': max_drawdown,
            'Calmar Ratio': calmar_ratio,
            'Risk-Free Rate': self.risk_free_rate
        }

        return profit, metrics

def run_strategy(ticker_symbol, start_date, end_date, strategy_types=['MACD'], show_metrics=False, show_graphs=False):
    ticker = Ticker(ticker_symbol, start_date, end_date)
    data = ticker.fetch_data()

    results = {}

    for strategy_type in strategy_types:
        if strategy_type == 'MACD':
            strategy = MACDStrategy(data.copy())
        elif strategy_type == 'BollingerBands':
            strategy = BollingerBandsStrategy(data.copy())
        else:
            raise ValueError("Unsupported strategy type")

        strategy_data = strategy.generate_signals()

        backtester = Backtester(strategy_data)
        profit, metrics = backtester.backtest()

        results[strategy_type] = {
            'profit': profit,
            'metrics': metrics
        }

        print(f"\nResults for {strategy_type} strategy:")
        print(f"  Final Portfolio Value: {profit['Final Portfolio Value']:.2f}")
        print(f"  Net Profit/Loss: {profit['Net Profit/Loss']:.2f}")

    # Always run the Buy and Hold strategy
    buy_and_hold_strategy = BuyAndHoldStrategy(data.copy())
    buy_and_hold_data = buy_and_hold_strategy.generate_signals()

    buy_and_hold_backtester = Backtester(buy_and_hold_data)
    buy_and_hold_profit, buy_and_hold_metrics = buy_and_hold_backtester.backtest()

    print("\nResults for Buy and Hold strategy:")
    print(f"  Final Portfolio Value: {buy_and_hold_profit['Final Portfolio Value']:.2f}")
    print(f"  Net Profit/Loss: {buy_and_hold_profit['Net Profit/Loss']:.2f}")

    return results

# Example of running the MACD and Bollinger Bands strategies separately

In [4]:
def analyze_metrics(metrics):
    analysis = {}

    if metrics['Sharpe Ratio'] < 1:
        analysis['Sharpe Ratio'] = "Poor"
    elif metrics['Sharpe Ratio'] < 2:
        analysis['Sharpe Ratio'] = "Acceptable"
    elif metrics['Sharpe Ratio'] < 3:
        analysis['Sharpe Ratio'] = "Good"
    else:
        analysis['Sharpe Ratio'] = "Excellent"

    if metrics['Sortino Ratio'] < 1:
        analysis['Sortino Ratio'] = "Poor"
    elif metrics['Sortino Ratio'] < 2:
        analysis['Sortino Ratio'] = "Acceptable"
    elif metrics['Sortino Ratio'] < 3:
        analysis['Sortino Ratio'] = "Good"
    else:
        analysis['Sortino Ratio'] = "Excellent"

    if metrics['Maximum Drawdown'] > 0.2:
        analysis['Maximum Drawdown'] = "Poor"
    elif metrics['Maximum Drawdown'] > 0.1:
        analysis['Maximum Drawdown'] = "Acceptable"
    else:
        analysis['Maximum Drawdown'] = "Good"

    if metrics['Calmar Ratio'] < 1:
        analysis['Calmar Ratio'] = "Poor"
    elif metrics['Calmar Ratio'] < 2:
        analysis['Calmar Ratio'] = "Acceptable"
    elif metrics['Calmar Ratio'] < 3:
        analysis['Calmar Ratio'] = "Good"
    else:
        analysis['Calmar Ratio'] = "Excellent"

    return analysis

def plot_results(data, ticker_symbol, title):
    plt.figure(figsize=(14, 9))

    plt.subplot(3, 1, 1)
    plt.plot(data['Close'], label='Close Price', color='black')
    plt.scatter(data.index[data['Buy_Signal'] == 1], data['Close'][data['Buy_Signal'] == 1], color='green', label='Buy Signal', marker='^', alpha=1)
    plt.scatter(data.index[data['Sell_Signal'] == 1], data['Close'][data['Sell_Signal'] == 1], color='red', label='Sell Signal', marker='v', alpha=1)
    plt.title(ticker_symbol + ' price with ' + title + ' Buy and Sell Signals')
    plt.legend()

    if 'MACD' in data.columns:
        plt.subplot(3, 1, 2)
        plt.plot(data['MACD'], label='MACD', color='blue')
        plt.plot(data['Signal'], label='Signal Line', color='red')
        plt.bar(data.index, data['Histogram'], label='Histogram', color='grey', alpha=0.3)
        plt.title('MACD Indicator')
        plt.legend()

    plt.subplot(3, 1, 3)
    plt.plot(data['Portfolio_Value'], label='Portfolio Value', color='blue')
    plt.title('Portfolio Value Over Time')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [3]:
results = run_strategy(
    ticker_symbol='AAPL',
    start_date='2020-01-01',
    end_date='2022-01-01',
    strategy_types=['MACD', 'BollingerBands'],
    show_metrics=False,
    show_graphs=False)

[*********************100%%**********************]  1 of 1 completed



Results for MACD strategy:
  Final Portfolio Value: 169573.76
  Net Profit/Loss: 69573.76

Results for BollingerBands strategy:
  Final Portfolio Value: 114714.61
  Net Profit/Loss: 14714.61

Results for Buy and Hold strategy:
  Final Portfolio Value: 236147.62
  Net Profit/Loss: 136147.62
